# Multilayer Perceptron

In [ ]:
#import libraries

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedShuffleSplit


In [ ]:
## Classification: MNIST

In [ ]:
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

In [ ]:
# Load dataset
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X.to_numpy()
y = y.to_numpy()

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
X_train = X_train.reshape(X_train.shape[0], 28*28)
X_test = X_test.reshape(X_test.shape[0], 28*28)

X_train /= 255
X_test /= 255

In [ ]:
print("Shape of training data", X_train.shape)
print("Shape of test data", X_test.shape)

print("Shape of training labels", y_train.shape)
print("Shape of test labels", y_test.shape)

In [ ]:
mlpc = MLPClassifier(hidden_layer_sizes=(128,))
mlpc.fit(X_train, y_train)

In [ ]:
cv_score = cross_val_score(mlpc, X_train, y_train.ravel(), cv =cv)
print("Training Accuracy: {:.2f} %".format(cv_score.mean()*100))

In [ ]:
mlpc.predict_proba(X_test[:5])

In [ ]:
y_pred = mlpc.predict(X_test)

In [ ]:
print("Train_accuracy: {:.2f} %".format(accuracy_score(y_train, mlpc.predict(X_train))*100))
print("Train_accuracy: {:.2f} %".format(accuracy_score(y_test, y_pred)*100))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion Matrix')
plt.show()

In [ ]:
CR = classification_report(y_test, y_pred)
print('Classification report \n')
print(CR)

In [ ]:
plt.plot(mlpc.loss_curve_, 'b-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss curve')

## Regression: California housing

In [ ]:
from sklearn.datasets import fetch_california_housing

from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_validate, ShuffleSplit

np.random.seed(306)
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

In [ ]:
X, y = fetch_california_housing(as_frame=True, return_X_y=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print("Shape of training data", X_train.shape)
print("Shape of test data", X_test.shape)

print("Shape of training labels", y_train.shape)
print("Shape of test labels", y_test.shape)

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()),
                    ('regressor', MLPRegressor(hidden_layer_sizes=(32)))])

cv_results = cross_validate(pipe, X_train, y_train, cv = cv,
                            scoring = "neg_mean_absolute_percentage_error",
                            return_train_score=True,
                            return_estimator=True,
                            n_jobs=2)

mlp_train_error = -1 * cv_results['train_score']
mlp_test_error = -1 * cv_results['test_score']

print(f"Mean absolute error of linear regression model on the train set:\n"
        f"{mlp_train_error.mean():.3f} +/- {mlp_train_error.std():.3f}")

print(f"Mean absolute error of linear regression model on the test set:\n"
        f"{mlp_test_error.mean():.3f} +/- {mlp_test_error.std():.3f}")

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
mean_absolute_percentage_error(y_train, pipe.predict(X_train))

In [ ]:
mean_absolute_percentage_error(y_test, pipe.predict(X_test))

In [ ]:
plt.plot(y_test, pipe.predict(X_test), 'b*')
plt.plot(y_test, y_test, 'g-')
plt.xlabel('actual')
plt.ylabel('predicted')